# Import packages

In [1]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam_v2
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import pickle
from keras.applications.vgg16 import VGG16 
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

# Import Data

In [2]:
path = "../input/imgssigns/myData"
labelFile = '../input/labelssigns/labels.csv'
batch_size = 100
epochs=40
H = 32
W = 32

In [ ]:
i = 0
images = []
classNo = []
List = os.listdir(path)
print("Nombre de classes:",len(List))
noOfClasses=len(List)
for x in range (0,len(List)):
    imgList = os.listdir(path+"/"+str(i))
    for m in imgList:
        img = cv2.imread(path+"/"+str(i)+"/"+m,0)
        images.append(img)
        classNo.append(i)
    i +=1
print("fin d'importation")
images = np.array(images)
classNo = np.array(classNo)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(images, classNo, test_size=0.2)

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
X_train, X_valid = X_train/255, X_valid/255

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_valid=X_valid.reshape(X_valid.shape[0],X_valid.shape[1],X_valid.shape[2],1)

In [ ]:
X_train.shape, X_valid.shape

# Deep Learning

In [ ]:
model = Sequential([
    Conv2D(32,(5,5),activation = 'relu', input_shape = (H,W,1)),
    Conv2D(32,(5,5),activation = 'relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,(3,3),activation = 'relu',padding='Same'),
    Conv2D(64,(3,3),activation = 'relu',padding='Same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(noOfClasses,activation='softmax')
])
model.summary()

In [ ]:
dataGen= ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10)

In [ ]:
dataGen.fit(X_train)

In [ ]:
y_train_c = to_categorical(y_train,noOfClasses)
y_valid_c = to_categorical(y_valid,noOfClasses)

In [ ]:
y_train_c.shape, y_valid_c.shape

In [ ]:
optm = Adam(
    learning_rate = 0.0001,
    epsilon = 0.00001,
    name='Adam'
)

In [ ]:
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
history = model.fit_generator(dataGen.flow(X_train,y_train_c,batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(X_valid,y_valid_c))

In [ ]:
score = model.evaluate(X_valid,y_valid_c)
print('loss : ',score[0])
print('accr : ',score[1])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'y', label="training")
plt.plot(epochs, val_loss, 'r', label="validation")
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'y', label="training")
plt.plot(epochs, val_acc, 'r', label="validation")
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
testImg = cv2.imread('../input/imgtest5/light.png',0)

In [ ]:
plt.imshow(testImg)

In [ ]:
tImg = cv2.resize(testImg, (H,W))
tImg = tImg/255
tImg = tImg.reshape(1,H,W,1)

In [ ]:
tImg.shape

In [ ]:
pred = model.predict(tImg)
probabilityValue =np.amax(pred)

In [ ]:
probabilityValue

In [ ]:
list(pred[0]).index(probabilityValue)

In [ ]:
model_json = model.to_json()
with open("modelDL.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("modelDL.h5")

In [ ]:
json_file = open('./modelDL.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('./modelDL.h5')

In [ ]:
loaded_model.compile(loss='binary_crossentropy', optimizer=optm, metrics=['accuracy'])
score = loaded_model.evaluate(X_valid,y_valid_c, verbose=0)

In [ ]:
score

In [ ]:
testImg1 = cv2.imread('../input/imgtest4/right.png',0)
plt.imshow(testImg1)

In [ ]:
tImg1 = cv2.resize(testImg1, (H,W))
tImg1 = tImg1/255
tImg1 = tImg1.reshape(1,H,W,1)
tImg1.shape

In [ ]:
pred1 = loaded_model.predict(tImg1)
probabilityValue1 =np.amax(pred1)

In [ ]:
list(pred1[0]).index(probabilityValue1)

# Deep Learning with Machine Learning

In [ ]:
i = 0
images = []
classNo = []
List = os.listdir(path)
print("Nombre de classes:",len(List))
noOfClasses=len(List)
for x in range (0,len(List)):
    imgList = os.listdir(path+"/"+str(i))
    for m in imgList:
        img = cv2.imread(path+"/"+str(i)+"/"+m)
        images.append(img)
        classNo.append(i)
    i +=1
print("fin d'importation")
images = np.array(images)
classNo = np.array(classNo)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(images, classNo, test_size=0.2)

In [ ]:
X_train, X_valid = X_train/255, X_valid/255

In [ ]:
y_train_c = to_categorical(y_train,noOfClasses)
y_valid_c = to_categorical(y_valid,noOfClasses)

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=X_train[0].shape)

In [ ]:
vgg.summary()

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
vgg.summary()

In [ ]:
X_train.shape

In [ ]:
getFeatures = vgg.predict(X_train)
Features_X_Train = getFeatures.reshape(getFeatures.shape[0],-1)
Features_X_Train.shape

In [ ]:
getFeaturesV = vgg.predict(X_valid)
Features_X_valid = getFeaturesV.reshape(getFeaturesV.shape[0],-1)
Features_X_valid.shape

In [ ]:
prepo = PCA(n_components=450)
Features_X_Train_N = prepo.fit_transform(Features_X_Train)
Features_X_valid_N = prepo.transform(Features_X_valid)

In [ ]:
mlp = MLPClassifier()

In [ ]:
%time mlp.fit(Features_X_Train_N, y_train)
print(mlp.score(Features_X_valid_N, y_valid))

In [ ]:
pickle.dump(mlp, open('./modelML', 'wb'))

In [ ]:
mlmodel = pickle.load(open('./modelML', 'rb'))
result = mlmodel.score(Features_X_valid_N, y_valid)
print(result)

In [ ]:
tImg1 = cv2.imread('../input/imgtest4/right.png')/255
tImg1 = cv2.resize(tImg1, (H,W))
tImg1 = tImg1.reshape(1,H,W,3)
print(tImg1.shape)
feat = vgg.predict(tImg1)
features = feat.reshape(feat.shape[0],-1)
featurespca = prepo.transform(features)
pred = mlp.predict(featurespca)

In [ ]:
pred[0]